This code creates price variance to be used in a VaR model

In [1]:
import pandas as pd
import numpy as np

In [ ]:
#Reading prices csv and adding to dataframek
df = pd.read_csv("instrumentprices.csv")
#Sort the dataframe on instrument and date
df = df.sort_values(['instrumentsymbol', 'tradedate'])

# Clean the data. Remove those with "To be deleted". This should remove two instruments
df = df[~df["instrumentname"].str.contains("deleted", case=False, na=False)]
df


#Alternative df = df[~df["instrumentsymbol"].isin(["yyyHKD", "xxxHKD"])]

,tradedate,instrumentsymbol,instrumentname,bidprice,AskPrice,midprice,assetclass,instrumenttypename,instrumentsubtypename,underlyingisin
0,2025-01-01,AUDCAD,AUDCAD,0.88565,0.89180,0.890065,CU,FX Spot,FXSpot,NaN
194,2025-01-02,AUDCAD,AUDCAD,0.89225,0.89505,0.893869,CU,FX Spot,FXSpot,NaN
388,2025-01-03,AUDCAD,AUDCAD,0.89668,0.90014,0.898300,CU,FX Spot,FXSpot,NaN
582,2025-01-04,AUDCAD,AUDCAD,0.89668,0.90014,0.898300,CU,FX Spot,FXSpot,NaN
776,2025-01-05,AUDCAD,AUDCAD,0.89668,0.90014,0.898300,CU,FX Spot,FXSpot,NaN
...,...,...,...,...,...,...,...,...,...,...
62328,2025-11-15,ZWFUSD,Wheat / USD,539.82000,539.92000,539.870000,NaN,FX Spot,FXSoft,NaN
62525,2025-11-16,ZWFUSD,Wheat / USD,539.82000,539.92000,539.870000,NaN,FX Spot,FXSoft,NaN
62722,2025-11-17,ZWFUSD,Wheat / USD,558.07000,558.17000,558.120000,NaN,FX Spot,FXSoft,NaN
62919,2025-11-18,ZWFUSD,Wheat / USD,558.34000,558.44000,558.390000,NaN,FX Spot,FXSoft,NaN


In [15]:
df['pct_change'] = df.groupby('instrumentsymbol')['midprice'].pct_change() * 100
df

C:\Users\oscar\AppData\Local\Temp\ipykernel_33860\2872555613.py:1: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df['pct_change'] = df.groupby('instrumentsymbol')['midprice'].pct_change() * 100


,tradedate,instrumentsymbol,instrumentname,bidprice,AskPrice,midprice,assetclass,instrumenttypename,instrumentsubtypename,underlyingisin,pct_change
0,2025-01-01,AUDCAD,AUDCAD,0.88565,0.89180,0.890065,CU,FX Spot,FXSpot,NaN,NaN
194,2025-01-02,AUDCAD,AUDCAD,0.89225,0.89505,0.893869,CU,FX Spot,FXSpot,NaN,0.427497
388,2025-01-03,AUDCAD,AUDCAD,0.89668,0.90014,0.898300,CU,FX Spot,FXSpot,NaN,0.495654
582,2025-01-04,AUDCAD,AUDCAD,0.89668,0.90014,0.898300,CU,FX Spot,FXSpot,NaN,0.000000
776,2025-01-05,AUDCAD,AUDCAD,0.89668,0.90014,0.898300,CU,FX Spot,FXSpot,NaN,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
62328,2025-11-15,ZWFUSD,Wheat / USD,539.82000,539.92000,539.870000,NaN,FX Spot,FXSoft,NaN,0.000000
62525,2025-11-16,ZWFUSD,Wheat / USD,539.82000,539.92000,539.870000,NaN,FX Spot,FXSoft,NaN,0.000000
62722,2025-11-17,ZWFUSD,Wheat / USD,558.07000,558.17000,558.120000,NaN,FX Spot,FXSoft,NaN,3.380443
62919,2025-11-18,ZWFUSD,Wheat / USD,558.34000,558.44000,558.390000,NaN,FX Spot,FXSoft,NaN,0.048377


In [16]:
#This finds the values at 5th, 95th, and absolute 95th percentile.
stats = df.groupby('instrumentsymbol')['pct_change'].agg(
    p05=lambda x: x.quantile(0.05),
    p95=lambda x: x.quantile(0.95),
    abs_p95=lambda x: x.abs().quantile(0.95)
)
print(stats)


                       p05       p95   abs_p95
instrumentsymbol                              
AUDCAD           -0.477970  0.520061  0.670574
AUDCHF           -0.791901  0.635231  0.927500
AUDCNH           -0.595165  0.612625  0.759970
AUDDKK           -0.581559  0.592538  0.799457
AUDHKD                 NaN       NaN       NaN
...                    ...       ...       ...
ZARJPY           -1.105956  1.021444  1.443333
ZARMXN           -0.772986  0.791142  1.111307
ZCFUSD           -1.507699  1.430159  2.083040
ZSFUSD           -1.178305  1.482250  1.762925
ZWFUSD           -1.763345  2.193337  2.625347

[195 rows x 3 columns]


In [18]:
#This find the confidence intervals assuming normal distribution

stats = df.groupby('instrumentsymbol')['pct_change'].agg(['mean', 'std', 'count'])

z = 1.96

stats['ci_lower'] = stats['mean'] - z * stats['std'] / np.sqrt(stats['count'])
stats['ci_upper'] = stats['mean'] + z * stats['std'] / np.sqrt(stats['count'])

print(stats)


                      mean       std  count  ci_lower  ci_upper
instrumentsymbol                                               
AUDCAD            0.007700  0.388147    322 -0.034696  0.050096
AUDCHF           -0.021727  0.571577    322 -0.084159  0.040704
AUDCNH            0.007175  0.445917    322 -0.041531  0.055881
AUDDKK           -0.017514  0.449099    322 -0.066567  0.031540
AUDHKD                 NaN       NaN      0       NaN       NaN
...                    ...       ...    ...       ...       ...
ZARJPY            0.028637  0.683025    322 -0.045968  0.103241
ZARMXN           -0.008246  0.501628    322 -0.063037  0.046545
ZCFUSD           -0.004766  0.941122    322 -0.107562  0.098029
ZSFUSD            0.016818  0.756090    322 -0.065767  0.099403
ZWFUSD            0.007925  1.135280    322 -0.116078  0.131927

[195 rows x 5 columns]
